In [1]:
import pandas as pd
import numpy as np
import os
import time, datetime
import matplotlib.pyplot as plt

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#2020-11-02 18:51:24.674845
#67884674845
def get_time_stamp(time):
    hh = int(time[11:11+2])
    mm = int(time[14:14+2])
    ii = int(time[17:17+2])
    res = hh*3600+mm*60+ii
    for i in range(20,len(time)):
        res=res*10+ord(time[i])-ord('0')
    return res

In [3]:
train_data_c = pd.read_csv('CCF2021_run_record_c_Train.csv')
train_data_p = pd.read_csv('CCF2021_run_record_p_Train.csv')
train_data_label = pd.read_csv('CCF2021_run_label_Train.csv')
test_data_c = pd.read_csv('CCF2021_run_record_c_EvalA.csv')
test_data_p = pd.read_csv('CCF2021_run_record_p_EvalA.csv')
train_data_pre = pd.read_csv('train_presort.csv')
test_data_pre = pd.read_csv('test_presort.csv')

In [4]:
label_p_id = train_data_label['人员编号']
label_c_id = train_data_label['特征码']
answer = {label_p_id[i]:label_c_id[i] for i in range(len(label_p_id))}

In [5]:
%%time
train_data_c.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)
train_data_p.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)
test_data_c.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)
test_data_p.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)

CPU times: user 358 ms, sys: 6.92 ms, total: 365 ms
Wall time: 368 ms


In [6]:
%%time
train_data_c['time'] = train_data_c['发现时间'].apply(lambda x:get_time_stamp(x))
train_data_p['time'] = train_data_p['发现时间'].apply(lambda x:get_time_stamp(x))
test_data_c['time'] = test_data_c['发现时间'].apply(lambda x:get_time_stamp(x))
test_data_p['time'] = test_data_p['发现时间'].apply(lambda x:get_time_stamp(x))

CPU times: user 1.33 s, sys: 14.9 ms, total: 1.35 s
Wall time: 1.35 s


In [7]:
from math import radians, cos, sin, asin, sqrt

#公式计算两点间距离（m）

def geodistance(lng1,lat1,lng2,lat2):
    lng1,lat1,lng2,lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=round(distance/1000,3)
    return distance

In [21]:
%%time
c_map = {}
p_map = {}
c_ids = []
p_ids = []
cid_len = {}

speed_p = {}
speed_c = {}
def init(data_c,data_p,data_pre):
    global c_ids,p_ids,c_map,p_map,cid_len
    global speed_p,speed_c
    speed_p = {}
    speed_c = {}  

    c_map = {}
    p_map = {}
    c_ids = data_pre['c_id'].tolist()
    p_ids = data_pre['p_id'].tolist()    
    cid_len = {}
    dfs = data_c.groupby(['特征码','点位编号'])
    for df in dfs:
        c_map[df[0]] = df[1].values.tolist()

    dfs = data_c.groupby(['特征码'])
    for df in dfs:
        c_map[df[0]] = df[1].values.tolist()
    
    # speed_c
    for df in dfs: 
        speed = []
        cid = df[0] 
        # print(df[1].columns)
        pos = df[1]['点位编号'].tolist()
        lng = df[1]['经度'].tolist()
        lat = df[1]['纬度'].tolist()
        tt = df[1]['time'].tolist()
        for i in range(1,len(pos)):
            if pos[i] != pos[i-1]:
                dis = geodistance(lng[i],lat[i],lng[i-1],lat[i-1])
                speed.append(abs(dis/(tt[i]-tt[i-1])))
        speed.sort()
        speed_c[cid] = [0,0,0]
        speed_c[cid][2] = len(speed)
        if len(speed) != 0:
          speed_c[cid][0] = speed[len(speed)//2]
          if len(speed)%2 == 0:
            speed_c[cid][0] = (speed[len(speed)//2] + speed[len(speed)//2-1])/2
          speed_c[cid][1] = speed[0]
        
        

    dfs = data_c.groupby(['点位编号'])
    for df in dfs:
        c_map[df[0]] = df[1].values.tolist()

    dfs = data_p.groupby(['人员编号'])
    for df in dfs:
        p_map[df[0]] = df[1].values.tolist()

    # speed_p
    for df in dfs: 
        speed = []
        pid = df[0] 
        pos = df[1]['点位编号'].tolist()
        lng = df[1]['经度'].tolist()
        lat = df[1]['纬度'].tolist()
        tt = df[1]['time'].tolist()
        for i in range(1,len(pos)):
            if pos[i] != pos[i-1]:
                dis = geodistance(lng[i],lat[i],lng[i-1],lat[i-1])
                speed.append(abs(dis/(tt[i]-tt[i-1])))
        speed.sort()
        speed_p[pid] = [0,0,0]
        speed_p[pid][2] = len(speed)
        if len(speed) != 0:
          speed_p[pid][0] = speed[len(speed)//2]
          if len(speed)%2 == 0:
              speed_p[pid][0] = (speed[len(speed)//2] + speed[len(speed)//2-1])/2
          speed_p[1] = speed[0]


    for c_id in c_ids:
        cid_len[c_id] = len(c_map[c_id])
    print(len(c_ids),len(p_ids))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


In [9]:
cid_value_number = {}
def add_values(time,d_id,cid,iters=3000):
    global c_ids,p_ids,c_map,p_map,cid_len,cid_value_number
    if c_map.get((cid,d_id),-1)==-1:
        return
    n = len(c_map[(cid,d_id)])
    data_d_list = c_map[(cid,d_id)]
    lb = 0
    ub = n-1
    ans=-1
    while ub>=lb:
        mid = (lb+ub) // 2
        if data_d_list[mid][5]<=time:
            ans=mid
            lb=mid+1
        else:
            ub=mid-1
    if ans!=-1:
        value = time - data_d_list[ans][5]
        cid_value_number[cid][0]+=value
        cid_value_number[cid][1]+=1

In [10]:
def lcs(a,b):
    lena=len(a)
    lenb=len(b)
    c=[[0 for i in range(lenb+1)] for j in range(lena+1)]
    for i in range(lena):
        for j in range(lenb):
            if a[i]==b[j]:
                c[i+1][j+1]=c[i][j]+1
            elif c[i+1][j]>c[i][j+1]:
                c[i+1][j+1]=c[i+1][j]
            else:
                c[i+1][j+1]=c[i][j+1]
    return c[lena][lenb]

In [11]:
def get_unique_d_feature(c_id,p_id):
    unique_data_c_list = []
    unique_data_p_list = []
    data_c_list = c_map[c_id]
    data_p_list = p_map[p_id]
    for i in range(len(data_c_list)):
        if i>0 and data_c_list[i][0] == data_c_list[i-1][0]:
            continue
        unique_data_c_list.append(data_c_list[i])
    for i in range(len(data_p_list)):
        if i>0 and data_p_list[i][0] == data_p_list[i-1][0]:
            continue
        unique_data_p_list.append(data_p_list[i])
    unique_pp_number = 0
    unique_average_det_time = 0.0
    unique_lcs = 0
    Min_time = 1e30
    for i in range(len(unique_data_p_list)):
        for j in range(len(unique_data_c_list)-1,-1,-1):
            if unique_data_c_list[j][0] == unique_data_p_list[i][0] and unique_data_c_list[j][5] < unique_data_p_list[i][5]:
                unique_pp_number += 1
                unique_average_det_time += unique_data_c_list[j][5] - unique_data_p_list[i][5]
                Min_time = min(Min_time,-unique_data_c_list[j][5] + unique_data_p_list[i][5])
                break
    a = []
    b = []
    for i in range(len(unique_data_p_list)):
        a.append(unique_data_p_list[i][0])
    for i in range(len(unique_data_c_list)):
        b.append(unique_data_c_list[i][0])
    unique_lcs = lcs(a,b)
    if unique_pp_number == 0:
        return (unique_pp_number,1e20,unique_lcs,1e30)
    return (unique_pp_number,-unique_average_det_time/(unique_pp_number+0.001),unique_lcs,Min_time)

In [25]:
def work(data_c,data_p,data_pre,name):
    global c_ids,p_ids,c_map,p_map,cid_len,answer,cid_value_number
    global speed_p,speed_c
    print(data_c.shape,data_p.shape,data_pre.shape)
    init(data_c,data_p,data_pre)
    if name=='train':
        label = data_pre['label']
    pp_number = []
    average_det_time = []
    cid_number = []
    unique_pp_number = []
    unique_average_det_time = []
    unique_lcs = []
    
    # 新增
    p_number = []
    pid_number = []
    c_speed = []
    p_speed = []
    dis_speed = []
    c_speed_min = []
    p_speed_min = []
    dis_speed_min = []
    p_speed_change = []
    c_speed_change = []
    dis_change = []
    
    #新增
    Min_time = []

    for i in range(len(p_ids)):
        p_id = p_ids[i]
        c_id = c_ids[i]
        cid_value_number = {}
        cid_value_number.setdefault(c_id,[0,0])
        data_p_list = p_map[p_id]
        for j in range(len(data_p_list)):
            add_values(data_p_list[j][5],data_p_list[j][0],c_id)
        
        pp_number.append(cid_value_number[c_id][1]/len(data_p_list))
        average_det_time.append((cid_value_number[c_id][0]+0.0000)/(cid_value_number[c_id][1]+0.0000))
        cid_number.append(cid_len[c_id])
        tmp = get_unique_d_feature(c_id,p_id)
        unique_pp_number.append(tmp[0])
        unique_average_det_time.append(tmp[1])
        unique_lcs.append(tmp[2])
        Min_time.append(tmp[3])
        p_number.append(cid_value_number[c_id][1])
        pid_number.append(len(data_p_list))
        p_speed.append(speed_p[p_id][0])
        c_speed.append(speed_c[c_id][0])
        dis_speed.append(abs(speed_p[p_id][0] - speed_c[c_id][0]))
        p_speed_min.append(speed_p[p_id][1])
        c_speed_min.append(speed_c[c_id][1])
        dis_speed_min.append(abs(speed_p[p_id][1] - speed_c[c_id][1]))
        p_speed_change.append(speed_p[p_id][2])
        c_speed_change.append(speed_c[c_id][2])
        dis_change.append(speed_p[p_id][2]-speed_c[c_id][2])


    
    print(name,len(c_ids),len(p_ids),len(pp_number),len(average_det_time),len(cid_number))
    feature = pd.DataFrame({'c_id':c_ids,
                            'p_id':p_ids,
                            'pp_number':pp_number,
                            'average_det_time':average_det_time,
                            'cid_number':cid_number,
                            'unique_pp_number':unique_pp_number,
                            'unique_average_det_time':unique_average_det_time,
                            'unique_lcs':unique_lcs,
                            'p_number':p_number,
                            'pid_number':pid_number,
                            'p_speed':p_speed,
                            'c_speed':c_speed,
                            'dis_speed':dis_speed,
                            'p_speed_min':p_speed_min,
                            'c_speed_min':c_speed_min,
                            'dis_speed_min':dis_speed_min,
                            'Min_time':Min_time,
                            'c_speed_change':c_speed_change,
                            'p_speed_change':p_speed_change,
                            'dis_change':dis_change,
                            })
    if name=='train':
        feature['label'] = label
    feature.to_csv(name+'_feature_new.csv',index=False)

In [26]:
work(train_data_c,train_data_p,train_data_pre,'train')
work(test_data_c,test_data_p,test_data_pre,'test')

(237023, 6) (14028, 6) (398338, 3)
398338 398338
train 398338 398338 398338 398338 398338
(146176, 6) (8386, 6) (196382, 2)
196382 196382
test 196382 196382 196382 196382 196382
